In [0]:
import pandas as pd
import matplotlib as plt

In [0]:
csv_data = pd.read_csv("https://raw.githubusercontent.com/CSafewright/CS440_autism_datasets/refs/heads/main/autism_prevalence_studies_20250928.csv")
df = pd.DataFrame(csv_data)

Dropping unneeded columns

In [0]:
df.drop(['Title','Link to Publication', 'CDC Calculated Values',
         'Adaptive Score <70 (%)', 'Diagnosis Age Range (months)',
         'Diagnosis Median Age (months)', 'Diagnosis Mean Age (months)',
         'Non-Verbal or Minimally Verbal (%)', 'Percentage of Individual Co-occurring Conditions',
         'IQ Score <70 (%)', 'Non-Hispanic White:Hispanic Prevalence Ratio',
         'White:Black Prevalence Ratio', 'Autism Types Included',
         'Case Criterion', 'Confidence Interval (CI)'], axis=1, inplace=True)

Dropping study where 18 to 64 was the age range. Not important to us, we are looking at autism in children.

In [0]:
df = df[df['Age Range'] != '18 to 64']

In [0]:
df['Area(s)'] = df['Area(s)'].fillna('Unknown')

In [0]:
df.count()

Author                               206
Year Published                       206
Country                              206
Area(s)                              206
Age Range                            201
Study Years                          184
Case Identification Method           206
Sample Size                          191
Number of Cases                      191
ASD Prevalence Estimate per 1,000    206
Male:Female Sex Ratio                138
dtype: int64

Splitting 'Study Years' column and 'Age Range' column into two seperate columns each.  
When the study years only occured for one year, the year started and year ended column are the same.  
When the age range is only one age, the youngest and oldest age columns are the same.

In [0]:
# Split Study Years into start and end
df[['year_started', 'year_ended']] = (
    df['Study Years'].str.extract(r'(\d+)\s*-\s*(\d+)')
)

# If there are " & " cases instead of "-", handle them
mask_alt = df['Study Years'].str.contains('&', na=False)
df.loc[mask_alt, ['year_started', 'year_ended']] = (
    df.loc[mask_alt, 'Study Years'].str.extract(r'(\d+)\s*&\s*(\d+)')
)

df['year_started'] = df['year_started'].fillna(df['Year Published'])

# Fill missing end years with start years
df['year_ended'] = df['year_ended'].fillna(df['year_started'])

# Split Age Range
df[['youngest_age', 'oldest_age']] = (
    df['Age Range'].str.extract(r'(\d+)\s*to\s*(\d+)')
)

df['youngest_age'] = df['youngest_age'].fillna(df['Age Range'])

# Fill missing oldest_age with youngest_age
df['oldest_age'] = df['oldest_age'].fillna(df['youngest_age'])

# Drop original columns
#df = df.drop(columns=['Age Range', 'Study Years'])

# Convert to numeric
df[['year_started', 'year_ended', 'youngest_age', 'oldest_age', 'Sample Size', 'Number of Cases']] = (
    df[['year_started', 'year_ended', 'youngest_age', 'oldest_age', 'Sample Size', 'Number of Cases']].apply(pd.to_numeric, errors='coerce')
)

In [0]:
df.describe()

,Year Published,Sample Size,Number of Cases,"ASD Prevalence Estimate per 1,000",Male:Female Sex Ratio,year_started,year_ended,youngest_age,oldest_age
count,206.000000,8.000000,136.000000,206.000000,138.000000,206.000000,206.000000,200.000000,200.000000
mean,2009.266990,679.750000,211.448529,8.637704,4.081232,2006.635922,2008.033981,4.355000,11.710000
std,10.815146,207.304297,262.891467,8.207193,1.554231,11.674382,10.930102,3.004013,5.553386
min,1966.000000,374.000000,2.000000,0.077000,1.100000,1962.000000,1966.000000,0.000000,2.000000
25%,2005.250000,617.000000,34.750000,2.355000,3.000000,2002.250000,2004.000000,2.000000,8.000000
50%,2012.000000,707.500000,70.500000,6.200000,4.100000,2010.000000,2011.000000,5.000000,11.000000
75%,2017.000000,759.250000,301.250000,12.175000,4.775000,2015.000000,2016.000000,7.000000,17.000000
max,2022.000000,998.000000,987.000000,43.600000,9.900000,2022.000000,2022.000000,15.000000,30.000000


In [0]:
avg_youngest_age = df['youngest_age'].mean()
avg_oldest_age = df['oldest_age'].mean()
df['youngest_age'] = df['youngest_age'].fillna(avg_youngest_age)
df['oldest_age'] = df['oldest_age'].fillna(avg_oldest_age)

Changing empty spaces in Area(s), Sample Size, and Number of Cases columns to 'Unknown'

In [0]:
df['Area(s)'] = df['Area(s)'].fillna('Unknown')

In [0]:
df['Sample Size'].mean()

np.float64(679.75)

In [0]:
df['Number of Cases'].mean()
#df['Number of Cases'].fillna(df['Number of Cases'].mean(), inplace=True)']

np.float64(211.4485294117647)

In [0]:
df['ASD Prevalence Estimate per 1,000'].mean()

np.float64(8.637703883495146)

In [0]:
df.count()

Author                               206
Year Published                       206
Country                              206
Area(s)                              206
Age Range                            201
Study Years                          184
Case Identification Method           206
Sample Size                            8
Number of Cases                      136
ASD Prevalence Estimate per 1,000    206
Male:Female Sex Ratio                138
year_started                         206
year_ended                           206
youngest_age                         206
oldest_age                           206
dtype: int64

Male:Female Sex Ratio had 67% of cells. There were only 39 below 3.0 and 59 below 4.0. There were ~75 above 4.0.  
I felt it safe to assign the mean to of 4.07 to the empty cells.
This update changes the USA average from 3.95 to 3.999 (4.0)
For every other study outside of the US, the average changes from 4.114 to 4.1

In [0]:
mf_avg_ratio = df['Male:Female Sex Ratio'].mean()
df['Male:Female Sex Ratio'] = df['Male:Female Sex Ratio'].fillna(mf_avg_ratio)

In [0]:
df.describe()

,Year Published,Sample Size,Number of Cases,"ASD Prevalence Estimate per 1,000",Male:Female Sex Ratio,year_started,year_ended,youngest_age,oldest_age
count,206.000000,8.000000,136.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000
mean,2009.266990,679.750000,211.448529,8.637704,4.081232,2006.635922,2008.033981,4.355000,11.710000
std,10.815146,207.304297,262.891467,8.207193,1.270571,11.674382,10.930102,2.959726,5.471514
min,1966.000000,374.000000,2.000000,0.077000,1.100000,1962.000000,1966.000000,0.000000,2.000000
25%,2005.250000,617.000000,34.750000,2.355000,3.600000,2002.250000,2004.000000,2.000000,8.000000
50%,2012.000000,707.500000,70.500000,6.200000,4.081232,2010.000000,2011.000000,4.677500,11.000000
75%,2017.000000,759.250000,301.250000,12.175000,4.300000,2015.000000,2016.000000,6.000000,16.000000
max,2022.000000,998.000000,987.000000,43.600000,9.900000,2022.000000,2022.000000,15.000000,30.000000


In [0]:
df['Country'].unique()

array(['England', 'USA', 'Denmark', 'Japan', 'Sweden', 'Ireland',
       'Germany', 'Canada', 'France', 'Indonesia', 'Wales', 'Norway',
       'Finland', 'Iceland', 'United Kingdom', 'Australia', 'Spain',
       'China', 'Scotland', 'Portugal', 'United Arab Emirates',
       'Venezuela', 'Caribbean', 'Sri Lanka', 'Oman', 'Taiwan',
       'South Korea', 'Brazil', 'Iran', 'Bahrain', 'Israel', 'Uganda',
       'United Kindgom', 'United Kindgon', 'Ecuador', 'Nigeria',
       'Lebanon', 'Mexico', 'India', 'Poland', 'Nepal', 'Italy',
       'Bangladesh', 'Qatar', 'Vietnam', 'Basque Country, Spain',
       'Greece', 'Saudi Arabia', 'Korea'], dtype=object)

In [0]:
print(df[df['Country'] == 'USA'])

                                                Author  ...  oldest_age
1                                             Treffert  ...       12.00
12                                        Burd, et al.  ...       18.00
17                                       Ritvo, et al.  ...       12.00
28                                      Halfon, et al.  ...       17.00
30                                     Hillman, et al.  ...       15.00
36                                    Bertrand, et al.  ...       10.00
37                                       Croen, et al.  ...       12.00
39                             Yeargin-Allsopp, et al.  ...       10.00
40                                      Gurney, et al.  ...       17.00
43                                       Geier, et al.  ...       11.71
49                                 Newschaffer, et al.  ...        6.00
53                                   Blanchard, et al.  ...       17.00
59                                    Shattuck, et al.  ...     

In [0]:
df['oldest_age'].mean() - df['youngest_age'].mean()
print(df['youngest_age']).mean()
print(df['oldest_age']).mean()

0      8.0
1      2.0
2      2.0
3      5.0
4      0.0
      ... 
202    8.0
203    5.0
204    8.0
205    6.0
206    0.0
Name: youngest_age, Length: 206, dtype: float64


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-5147349549171067>, line 2
      1 df['oldest_age'].mean() - df['youngest_age'].mean()
----> 2 print(df['youngest_age']).mean()
      3 print(df['oldest_age']).mean()

AttributeError: 'NoneType' object has no attribute 'mean'

In [0]:
print(df[df['Male:Female Sex Ratio'] < 4.0])
df.describe()


In [0]:
import matplotlib.pyplot as plt
import numpy as np

plt.boxplot(df['Male:Female Sex Ratio'])